## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [14]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [15]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_38.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_38.csv')

In [16]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [17]:
Total_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 100 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   시술_당시_나이               256344 non-null  object 
 1   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 2   배란_자극_여부               256344 non-null  int64  
 3   배란_유도_유형               256344 non-null  object 
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불임원인여부_불명확             256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-n

In [18]:
Total_categorical_columns = [
    "시술_당시_나이",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [19]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [20]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

In [21]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.preprocessing import StandardScaler


In [22]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [23]:
# 데이터 표준화
scaler = StandardScaler()
Total_X_train = scaler.fit_transform(Total_X_train)
Total_X_test = scaler.transform(Total_X_test)

# PyTorch Dataset 생성
train_dataset = TabularDataset(torch.FloatTensor(Total_X_train), torch.FloatTensor(Total_y_train.values))  # .values 추가
test_dataset = TabularDataset(torch.FloatTensor(Total_X_test), torch.FloatTensor(Total_y_test.values))  # .values 추가

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # 첫 번째 층 노드 수 증가
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)  # 두 번째 층 노드 수 증가
        self.fc3 = nn.Linear(32, 16)  # 세 번째 층 추가
        self.fc4 = nn.Linear(16, 8)  # 네 번째 층 추가
        self.fc5 = nn.Linear(8, 1)  # 출력층
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.5)  # 드롭아웃 추가

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)  # 드롭아웃 적용
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)  # 드롭아웃 적용
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        return self.sigmoid(x)


In [25]:
# 하이퍼파라미터 설정
input_size = Total_X_train.shape[1]
epochs = 30
learning_rate = 0.001  # 필요시 학습률 조정 가능

# 모델, 손실 함수, 옵티마이저 정의
model = NeuralNetwork(input_size)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 모델 훈련
for epoch in range(epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [1/30], Loss: 0.7440
Epoch [2/30], Loss: 0.6394
Epoch [3/30], Loss: 0.6772
Epoch [4/30], Loss: 0.7148
Epoch [5/30], Loss: 0.5216
Epoch [6/30], Loss: 0.5512
Epoch [7/30], Loss: 0.6615
Epoch [8/30], Loss: 0.6476
Epoch [9/30], Loss: 0.6102
Epoch [10/30], Loss: 0.7129
Epoch [11/30], Loss: 0.5269
Epoch [12/30], Loss: 0.6473
Epoch [13/30], Loss: 0.4333
Epoch [14/30], Loss: 0.3709
Epoch [15/30], Loss: 0.5730
Epoch [16/30], Loss: 0.4609
Epoch [17/30], Loss: 0.3930
Epoch [18/30], Loss: 0.5190
Epoch [19/30], Loss: 0.3841
Epoch [20/30], Loss: 0.5748
Epoch [21/30], Loss: 0.5377
Epoch [22/30], Loss: 0.3402
Epoch [23/30], Loss: 0.5671
Epoch [24/30], Loss: 0.4805
Epoch [25/30], Loss: 0.5982
Epoch [26/30], Loss: 0.5695
Epoch [27/30], Loss: 0.3576
Epoch [28/30], Loss: 0.6590
Epoch [29/30], Loss: 0.3626
Epoch [30/30], Loss: 0.4970


In [26]:
from sklearn.metrics import roc_auc_score

# 모델 평가
model.eval()  # 평가 모드로 변경
with torch.no_grad():
    predictions = []
    true_labels = []
    for inputs, targets in test_loader:
        outputs = model(inputs).squeeze()
        predicted_probabilities = outputs.numpy()  # 확률로 변환
        predictions.extend(predicted_probabilities)
        true_labels.extend(targets.numpy())  # 실제 라벨 추가

# AUC 계산
auc_score = roc_auc_score(true_labels, predictions)
print(f'AUC Score: {auc_score:.4f}')


AUC Score: 0.7281


## 제출

----

.